In [ ]:
FOLDER_PATH = "/Users/nguyenhuuan/Desktop/Dart_KOREA/FirstPage/Menu"

import pandas as pd
import re
import numpy as np
import os
from tqdm import tqdm

def get_url(df):
    rcpNo = df["rcpNo"].iloc[0]
    dcmNo = df["dcmNo"].iloc[0]
    eleId = df["eleId"].iloc[0]
    offset = df["offset"].iloc[0]
    length = df["length"].iloc[0]
    dtd = df["dtd"].iloc[0]
    return "https://dart.fss.or.kr/" + f"/report/viewer.do?rcpNo={rcpNo}&dcmNo={dcmNo}&eleId={eleId}&offset={offset}&length={length}&dtd={dtd}"

df = pd.read_csv("/Users/nguyenhuuan/Desktop/Dart_KOREA/Metadata/filtered_2010_2023.csv")
df.shape

In [ ]:
BS_cols = ['2-1.연결재무상태표', '연결재무상태표', '2-1.연결재무상태표(대차대조표)', '가.연결재무상태표',
       '2-1.1연결재무상태표', '재무상태표', '-연결재무상태표', '가..연결재무상태표', '1)연결재무상태표']
CIC_cols = ['2-2.연결포괄손익계산서', '2-3.연결포괄손익계산서', '연결포괄손익계산서', '2-2.연결포괄손익계산서(손익계산서)',
       '나.연결포괄손익계산서', '포괄손익계산서', '2-2연결포괄손익계산서', '-연결포괄손익계산서', '2)연결포괄손익계산서']
IC_cols = ['2-2.연결손익계산서', '2-2.연결손익계산']

len(BS_cols), len(CIC_cols), len(IC_cols)

In [ ]:
list_data = []
for report_id in tqdm(df["report_id"]):
    df_menu = pd.read_csv(f"{FOLDER_PATH}/{report_id}.csv")
    try:
        chap3 = df_menu[(df_menu["node_level"] == 1) & (df_menu["text"] == "III.재무에관한사항")]
        assert len(chap3) == 1

        chap3_index = chap3.index[0]
        chap4_index = df_menu[(df_menu["node_level"] == 1) & (df_menu.index > chap3_index)].index[0]
        chap3_elements = df_menu[df_menu.index.isin(range(chap3_index, chap4_index))]
        assert len(chap3_elements) > 1

        sec2 = df_menu[(df_menu["node_level"] == 2) & (df_menu["text"] == "2.연결재무제표")]
        assert len(sec2) == 1 and chap3_index < sec2.index[0] and sec2.index[0] < chap4_index

        sec3 = df_menu[(df_menu["node_level"] == 2) & (df_menu["text"] == "3.연결재무제표주석")]
        assert len(sec3) == 1 and chap3_index < sec3.index[0] and sec3.index[0] < chap4_index \
            and sec2.index[0] < sec3.index[0]

        sec2_elements = df_menu[df_menu.index.isin(range(sec2.index[0], sec3.index[0]))]
        # assert len(sec2_elements) > 1
        assert len(sec2_elements) == 1
    except: continue

    # assert (sec2_elements.iloc[1:]["node_level"] == 3).all()
    data = {"report_id": report_id}
    # for i in range(1, len(sec2_elements)):
    #     text = sec2_elements.iloc[i]["text"]
    #     # try: data[text] += 1
    #     # except: data[text] = 1

    #     if text in BS_cols:
    #         name = "BS"
    #     elif text in CIC_cols:
    #         name = "CIC"
    #     elif text in IC_cols:
    #         name = "IC"
    #     else: continue

    #     assert name not in data.keys()
    #     data[name] = get_url(sec2_elements.iloc[i:i+1])
    data["url"] = get_url(sec2_elements)

    # assert "BS" in data.keys() and ("CIC" in data.keys() or "IC" in data.keys())
    list_data.append(data)

temp_df = pd.DataFrame(list_data)
temp_df

In [4]:
# temp_df.to_csv("df_consolidated_subsec.csv", index=False)
temp_df.to_csv("df_consolidated_sec2.csv", index=False)